## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [65]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [97]:
reviews_df = pd.DataFrame(columns=['seat_type','is_verified', 'rating','date_posted','review'])

base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 38
page_size = 100

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    # print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    
    for div in parsed_content.find_all('article', itemprop='review'):
        
        # review text
        text_content = div.find('div', class_='text_content').get_text() 
        
        # seat type
        header_td = div.find('td', class_='review-rating-header cabin_flown')
        if header_td:
            value_td = header_td.find_next_sibling('td', class_='review-value')
            if value_td:
                review_value = value_td.text.strip()
            else:
                review_value = None
        else:
            review_value = None
        
        # checking if verified review 
        is_verified=True if "Trip Verified" in text_content else False
        
        # Overall rating by customer
        if div.find('span', itemprop='ratingValue'):
            overall_rating = div.find('span', itemprop='ratingValue').get_text()
        else:
            overall_rating = None
        
        # date review was posted
        date_of_review = pd.to_datetime(div.find('time', itemprop="datePublished").get_text())
        
        reviews_df.loc[len(reviews_df)] = {'seat_type':review_value, 'review': text_content, 'is_verified' : is_verified, 'rating': overall_rating, 'date_posted': date_of_review}
        
reviews_df.head()
    

,seat_type,is_verified,rating,date_posted,review
0,Economy Class,True,1,2023-11-19,✅ Trip Verified | 4 Hours before takeoff we r...
1,Economy Class,True,3,2023-11-19,✅ Trip Verified | I recently had a delay on B...
2,Business Class,False,8,2023-11-16,"Not Verified | Boarded on time, but it took a..."
3,Economy Class,True,1,2023-11-16,"✅ Trip Verified | 5 days before the flight, w..."
4,Economy Class,False,1,2023-11-14,Not Verified | \r\nWe traveled to Lisbon for ...


In [101]:
reviews_df.to_csv("data/BA_reviews.csv", index=False)

Congratulations! Now you have your dataset for this task! The loops above collected 1000 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.